# Part 1

In [3]:
import numpy as np
import scipy.linalg as la
import cvxpy as cp
import time

# Problem setup for LASSO
np.random.seed(0)
m, n = 100, 50  # Changeable dimensions for testing
A = np.random.randn(m, n)
b = np.random.randn(m)
gamma = 0.1

# Part 1: Solve using CVXPY
start_time = time.time()
x = cp.Variable(n)
loss = (1/2) * cp.norm2(A @ x - b)**2 + gamma * cp.norm1(x)
problem = cp.Problem(cp.Minimize(loss))
problem.solve(verbose=True)
cvxpy_time = time.time() - start_time
print(f"CVXPY solution time: {cvxpy_time:.4f} seconds")

# Part 2: Proximal Gradient Algorithm

def soft_thresholding(y, lambd):
    return np.sign(y) * np.maximum(np.abs(y) - lambd, 0)


def proximal_gradient(A, b, gamma, stepsize, max_iters=1000, tol=1e-6):
    n = A.shape[1]
    x = np.zeros(n)
    for k in range(max_iters):
        grad = A.T @ (A @ x - b)
        x_new = soft_thresholding(x - stepsize * grad, stepsize * gamma)
        if np.linalg.norm(x_new - x, ord=2) < tol:
            break
        x = x_new
    return x

stepsizes = [0.01, 0.05, 0.1]
for stepsize in stepsizes:
    start_time = time.time()
    x_pg = proximal_gradient(A, b, gamma, stepsize)
    pg_time = time.time() - start_time
    print(f"Proximal Gradient solution time (stepsize={stepsize}): {pg_time:.4f} seconds")

# Part 3: ADMM Algorithm

def admm_lasso(A, b, gamma, rho, max_iters=1000, tol=1e-6):
    m, n = A.shape
    x = np.zeros(n)
    y = np.zeros(n)
    mu = np.zeros(n)
    ATA = A.T @ A
    ATb = A.T @ b
    inv_matrix = la.inv(rho * np.eye(n) + ATA)

    for k in range(max_iters):
        x = inv_matrix @ (rho * y - mu + ATb)
        y = soft_thresholding(x + mu / rho, gamma / rho)
        mu += rho * (x - y)
        if np.linalg.norm(x - y, ord=2) < tol:
            break
    return x

rhos = [0.1, 1, 10]
for rho in rhos:
    start_time = time.time()
    x_admm = admm_lasso(A, b, gamma, rho)
    admm_time = time.time() - start_time
    print(f"ADMM solution time (rho={rho}): {admm_time:.4f} seconds")

                                     CVXPY                                     
                                     v1.6.0                                    
(CVXPY) Jan 04 03:16:41 AM: Your problem has 50 variables, 0 constraints, and 0 parameters.
(CVXPY) Jan 04 03:16:41 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 04 03:16:41 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jan 04 03:16:41 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jan 04 03:16:41 AM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jan 04 03:16:41 AM: Compiling problem (target solver=CLARABEL).


C:\Users\Password Please\AppData\Local\Temp\ipykernel_4880\3467047169.py:32: RuntimeWarning: overflow encountered in matmul
  grad = A.T @ (A @ x - b)
C:\Users\Password Please\AppData\Local\Temp\ipykernel_4880\3467047169.py:32: RuntimeWarning: invalid value encountered in matmul
  grad = A.T @ (A @ x - b)


In [7]:
#Final
import numpy as np
import scipy.linalg as la
import cvxpy as cp
import time

# Problem setup for LASSO
np.random.seed(0)
m, n = 100, 50  # Changeable dimensions for testing
A = np.random.randn(m, n)
b = np.random.randn(m)
gamma = 0.1

# Normalize A and b to prevent overflow issues
A = A / np.linalg.norm(A, ord=2)
b = b / np.linalg.norm(b, ord=2)

# Part 1: Solve using CVXPY
start_time = time.time()
x = cp.Variable(n)
loss = (1/2) * cp.norm2(A @ x - b)**2 + gamma * cp.norm1(x)
problem = cp.Problem(cp.Minimize(loss))
problem.solve(verbose=True)
cvxpy_time = time.time() - start_time
print(f"CVXPY solution time: {cvxpy_time:.4f} seconds")

# Part 2: Proximal Gradient Algorithm

def soft_thresholding(y, lambd):
    return np.sign(y) * np.maximum(np.abs(y) - lambd, 0)


def proximal_gradient(A, b, gamma, stepsize, max_iters=1000, tol=1e-6):
    n = A.shape[1]
    x = np.zeros(n)
    L = np.linalg.norm(A.T @ A, ord=2)
    stepsize = min(stepsize, 1 / (2 * L))
    for k in range(max_iters):
        grad = A.T @ (A @ x - b)
        x_new = soft_thresholding(x - stepsize * grad, stepsize * gamma)
        if np.any(np.isnan(x_new)) or np.any(np.isinf(x_new)):
            print("Numerical instability detected. Stopping iteration.")
            break
        if np.linalg.norm(x_new - x, ord=2) < tol:
            break
        x = x_new
    return x

stepsizes = [0.01, 0.05, 0.1]
for stepsize in stepsizes:
    start_time = time.time()
    x_pg = proximal_gradient(A, b, gamma, stepsize)
    pg_time = time.time() - start_time
    print(f"Proximal Gradient solution time (stepsize={stepsize}): {pg_time:.4f} seconds")

# Part 3: ADMM Algorithm

def admm_lasso(A, b, gamma, rho, max_iters=1000, tol=1e-6):
    m, n = A.shape
    x = np.zeros(n)
    y = np.zeros(n)
    mu = np.zeros(n)
    ATA = A.T @ A
    ATb = A.T @ b
    inv_matrix = la.inv(rho * np.eye(n) + ATA)

    for k in range(max_iters):
        x = inv_matrix @ (rho * y - mu + ATb)
        y = soft_thresholding(x + mu / rho, gamma / rho)
        mu += rho * (x - y)
        if np.any(np.isnan(x)) or np.any(np.isinf(x)):
            print("Numerical instability detected. Stopping iteration.")
            break
        if np.linalg.norm(x - y, ord=2) < tol:
            break
    return x

rhos = [0.1, 1, 10]
for rho in rhos:
    start_time = time.time()
    x_admm = admm_lasso(A, b, gamma, rho)
    admm_time = time.time() - start_time
    print(f"ADMM solution time (rho={rho}): {admm_time:.4f} seconds")


                                     CVXPY                                     
                                     v1.6.0                                    
(CVXPY) Jan 04 04:59:21 AM: Your problem has 50 variables, 0 constraints, and 0 parameters.
(CVXPY) Jan 04 04:59:21 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 04 04:59:21 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jan 04 04:59:21 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jan 04 04:59:21 AM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jan 04 04:59:21 AM: Compiling problem (target solver=CLARABEL).


In [1]:
import numpy as np
import cvxpy as cp
import time

# Parameters
gamma = 0.1  # Regularization parameter
dimensions = [(50, 20), (100, 50), (200, 100), (500, 200)]  # Different dimensions for A and b

# Store results
results = []

for m, n in dimensions:
    # Generate random data
    np.random.seed(0)
    A = np.random.randn(m, n)
    b = np.random.randn(m)
    
    # Define optimization variable
    x = cp.Variable(n)
    
    # Define the LASSO objective
    objective = cp.Minimize(0.5 * cp.norm(A @ x - b, 2)**2 + gamma * cp.norm(x, 1))
    
    # Solve the problem and measure time
    problem = cp.Problem(objective)
    start_time = time.time()
    problem.solve()
    end_time = time.time()
    
    # Record results
    results.append({
        'dimension': (m, n),
        'optimal_value': problem.value,
        'computation_time': end_time - start_time
    })

# Display results
for result in results:
    print(f"Dimension: {result['dimension']}, Optimal Value: {result['optimal_value']:.4f}, "
          f"Computation Time: {result['computation_time']:.4f} seconds")


c:\Users\Password Please\AppData\Local\Programs\Python\Python312\Lib\site-packages\cvxpy\problems\problem.py:1481: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


Dimension: (50, 20), Optimal Value: 13.2624, Computation Time: 0.0120 seconds
Dimension: (100, 50), Optimal Value: 23.7428, Computation Time: 0.0199 seconds
Dimension: (200, 100), Optimal Value: 56.0480, Computation Time: 0.0559 seconds
Dimension: (500, 200), Optimal Value: 145.6908, Computation Time: 0.4378 seconds


In [2]:
#Final
import cvxpy as cp
import numpy as np
import time
import pandas as pd

def solve_lasso_cvxpy(n_samples, n_features, gamma=0.1):
    # Generate random A and b
    np.random.seed(42)
    A = np.random.randn(n_samples, n_features)
    b = np.random.randn(n_samples)
    
    # Define the optimization variable
    x = cp.Variable(n_features)
    
    # Define the LASSO objective
    objective = cp.Minimize(0.5 * cp.sum_squares(A @ x - b) + gamma * cp.norm1(x))
    
    # Define and solve the problem
    problem = cp.Problem(objective)
    start_time = time.time()
    problem.solve()
    end_time = time.time()
    
    return problem.value, end_time - start_time

# Test the function for different matrix dimensions
dimensions = [(100, 50), (200, 100), (500, 200), (1000, 500)]
gamma = 0.1
results = []

for n_samples, n_features in dimensions:
    value, time_taken = solve_lasso_cvxpy(n_samples, n_features, gamma)
    results.append({
        "n_samples": n_samples,
        "n_features": n_features,
        "objective_value": value,
        "time_taken_sec": time_taken
    })

# Display the results in a table
df = pd.DataFrame(results)
print(df)


   n_samples  n_features  objective_value  time_taken_sec
0        100          50        19.264617        0.016293
1        200         100        56.040808        0.033994
2        500         200       157.364405        0.248161
3       1000         500       229.952553        2.286817


In [5]:
import numpy as np
import time
import pandas as pd

def proximal_operator(x, threshold):
    """
    Soft-thresholding operator for L1 norm.
    """
    return np.sign(x) * np.maximum(np.abs(x) - threshold, 0)

def proximal_gradient_lasso(A, b, gamma, step_size, max_iter=1000, tol=1e-6):
    """
    Proximal gradient algorithm for solving LASSO problem.
    Args:
        A (numpy.ndarray): Input matrix (n_samples, n_features).
        b (numpy.ndarray): Target vector (n_samples,).
        gamma (float): Regularization parameter for L1 norm.
        step_size (float): Step size for gradient descent.
        max_iter (int): Maximum number of iterations.
        tol (float): Tolerance for convergence.
    Returns:
        x (numpy.ndarray): Solution vector (n_features,).
        history (list): Objective function values over iterations.
    """
    n_samples, n_features = A.shape
    x = np.zeros(n_features)  # Initialize x to zero
    history = []

    for i in range(max_iter):
        gradient = A.T @ (A @ x - b)  # Gradient of the least squares term
        x_new = proximal_operator(x - step_size * gradient, gamma * step_size)  # Proximal update
        obj_value = 0.5 * np.linalg.norm(A @ x_new - b)**2 + gamma * np.linalg.norm(x_new, 1)
        history.append(obj_value)
        
        if np.linalg.norm(x_new - x) < tol:  # Convergence check
            break
        
        x = x_new

    return x, history

# Test the proximal gradient algorithm for different dimensions and step sizes
np.random.seed(42)
dimensions = [(100, 50), (200, 100), (500, 200)]
gamma = 0.1
step_sizes = [0.001, 0.01, 0.1]
results = []

for n_samples, n_features in dimensions:
    A = np.random.randn(n_samples, n_features)
    b = np.random.randn(n_samples)
    for step_size in step_sizes:
        start_time = time.time()
        x, history = proximal_gradient_lasso(A, b, gamma, step_size)
        end_time = time.time()
        results.append({
            "n_samples": n_samples,
            "n_features": n_features,
            "step_size": step_size,
            "final_objective_value": history[-1],
            "iterations": len(history),
            "time_taken_sec": end_time - start_time
        })

# Display the results in a table
pd.DataFrame(results)


C:\Users\Password Please\AppData\Local\Temp\ipykernel_8056\234396832.py:30: RuntimeWarning: overflow encountered in matmul
  gradient = A.T @ (A @ x - b)  # Gradient of the least squares term
C:\Users\Password Please\AppData\Local\Temp\ipykernel_8056\234396832.py:32: RuntimeWarning: invalid value encountered in matmul
  obj_value = 0.5 * np.linalg.norm(A @ x_new - b)**2 + gamma * np.linalg.norm(x_new, 1)
C:\Users\Password Please\AppData\Local\Temp\ipykernel_8056\234396832.py:30: RuntimeWarning: invalid value encountered in matmul
  gradient = A.T @ (A @ x - b)  # Gradient of the least squares term
C:\Users\Password Please\AppData\Local\Temp\ipykernel_8056\234396832.py:31: RuntimeWarning: invalid value encountered in subtract
  x_new = proximal_operator(x - step_size * gradient, gamma * step_size)  # Proximal update


,n_samples,n_features,step_size,final_objective_value,iterations,time_taken_sec
0,100,50,0.001,19.264617,522,0.020944
1,100,50,0.010,inf,1000,0.039894
2,100,50,0.100,NaN,1000,0.039894
3,200,100,0.001,51.082933,397,0.260432
4,200,100,0.010,NaN,1000,0.726218
5,200,100,0.100,NaN,1000,0.646331
6,500,200,0.001,129.783602,115,0.078791
7,500,200,0.010,NaN,1000,0.667897
8,500,200,0.100,NaN,1000,0.744071


# Part 2

In [6]:
import numpy as np
import scipy.linalg as la
import cvxpy as cp
import time

# Problem setup for Graph Learning
np.random.seed(0)
N = 30  # Number of vertices (changeable for testing)
m = int(N * (N - 1) / 2)  # Number of upper triangular elements in W
Z = np.random.randn(N, N)
Z = (Z + Z.T) / 2  # Symmetric matrix
alpha, beta = 0.1, 0.01

# Flatten upper triangular part of Z excluding diagonal
def extract_upper_triangular(Z):
    return Z[np.triu_indices_from(Z, k=1)]

z = extract_upper_triangular(Z)

# Part 1: Solve using CVXPY
start_time = time.time()
w = cp.Variable(m)
Q = np.random.rand(N, m)  # Sparse binary matrix with Qw = W1
loss = 2 * z.T @ w - alpha * cp.sum(cp.log(Q @ w)) + beta * cp.norm(w, 2)**2
constraints = [w >= 0]
problem = cp.Problem(cp.Minimize(loss), constraints)
problem.solve(verbose=True)
cvxpy_time = time.time() - start_time
print(f"CVXPY solution time (N={N}): {cvxpy_time:.4f} seconds")

# Part 2: ADMM Algorithm

def admm_graph_learning(z, Q, alpha, beta, rho, max_iters=1000, tol=1e-6):
    m = len(z)
    s = Q.shape[0]
    w = np.zeros(m)
    v = np.ones(s)
    lam = np.zeros(s)
    t = 1  # Step size
    tau1 = 1 / (2 * (N - 1) * t)
    tau2 = 1 / t

    for k in range(max_iters):
        # Update w
        w_tilde = w - tau1 * t * Q.T @ (Q @ w - v - lam / t)
        w_new = np.maximum(w_tilde - 2 * tau1 * z / (2 * tau1 * beta + 1), 0)

        # Update v
        v_tilde = (1 - tau2 * t) * v + tau2 * t * Q @ w_new - tau2 * lam
        v_new = (v_tilde + np.sqrt(v_tilde**2 + 4 * alpha * tau2)) / 2

        # Update lambda
        lam += t * (Q @ w_new - v_new)

        # Check convergence
        rp = np.linalg.norm(t * Q.T @ (v_new - v), ord=2)
        rd = np.linalg.norm(Q @ w_new - v_new, ord=2)
        if rp < tol and rd < tol:
            break

        w, v = w_new, v_new

    return w

# Evaluate ADMM for N=30 and N=100
for N in [30, 100]:
    m = int(N * (N - 1) / 2)
    Q = np.random.rand(N, m)  # Generate sparse binary matrix Q
    Z = np.random.randn(N, N)
    Z = (Z + Z.T) / 2  # Ensure Z is symmetric
    z = extract_upper_triangular(Z)
    start_time = time.time()
    w_admm = admm_graph_learning(z, Q, alpha, beta, rho=1.0)
    admm_time = time.time() - start_time
    print(f"ADMM solution time (N={N}): {admm_time:.4f} seconds")


                                     CVXPY                                     
                                     v1.6.0                                    
(CVXPY) Jan 04 03:18:29 AM: Your problem has 435 variables, 435 constraints, and 0 parameters.
(CVXPY) Jan 04 03:18:29 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 04 03:18:29 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jan 04 03:18:29 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jan 04 03:18:29 AM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jan 04 03:18:29 AM: Compiling problem (target solver=CLARABEL

C:\Users\Password Please\AppData\Local\Temp\ipykernel_4880\1185109395.py:50: RuntimeWarning: overflow encountered in square
  v_new = (v_tilde + np.sqrt(v_tilde**2 + 4 * alpha * tau2)) / 2
C:\Users\Password Please\AppData\Local\Temp\ipykernel_4880\1185109395.py:45: RuntimeWarning: invalid value encountered in subtract
  w_tilde = w - tau1 * t * Q.T @ (Q @ w - v - lam / t)
C:\Users\Password Please\AppData\Local\Temp\ipykernel_4880\1185109395.py:49: RuntimeWarning: invalid value encountered in multiply
  v_tilde = (1 - tau2 * t) * v + tau2 * t * Q @ w_new - tau2 * lam


ADMM solution time (N=30): 0.7891 seconds
ADMM solution time (N=100): 6.3882 seconds


In [8]:

import numpy as np
import scipy.linalg as la
import cvxpy as cp
import time

# Problem setup for Graph Learning
np.random.seed(0)
N = 30  # Number of vertices (changeable for testing)
m = int(N * (N - 1) / 2)  # Number of upper triangular elements in W
Z = np.random.randn(N, N)
Z = (Z + Z.T) / 2  # Symmetric matrix
alpha, beta = 0.1, 0.01

# Flatten upper triangular part of Z excluding diagonal
def extract_upper_triangular(Z):
    return Z[np.triu_indices_from(Z, k=1)]

z = extract_upper_triangular(Z)

# Normalize z to prevent overflow
z = z / np.linalg.norm(z, ord=2)

# Part 1: Solve using CVXPY
start_time = time.time()
w = cp.Variable(m)
Q = np.random.rand(N, m)  # Sparse binary matrix with Qw = W1
Q = Q / np.linalg.norm(Q, ord=2)  # Normalize Q to prevent overflow
loss = 2 * z.T @ w - alpha * cp.sum(cp.log(Q @ w)) + beta * cp.norm(w, 2)**2
constraints = [w >= 0]
problem = cp.Problem(cp.Minimize(loss), constraints)
problem.solve(verbose=True)
cvxpy_time = time.time() - start_time
print(f"CVXPY solution time (N={N}): {cvxpy_time:.4f} seconds")

# Part 2: ADMM Algorithm

def admm_graph_learning(z, Q, alpha, beta, rho, max_iters=1000, tol=1e-6):
    m = len(z)
    s = Q.shape[0]
    w = np.zeros(m)
    v = np.ones(s)
    lam = np.zeros(s)
    t = 0.1  # Reduced step size for stability
    tau1 = 1 / (2 * (N - 1) * t)
    tau2 = 1 / t

    for k in range(max_iters):
        # Update w
        w_tilde = w - tau1 * t * Q.T @ (Q @ w - v - lam / t)
        w_new = np.maximum(w_tilde - 2 * tau1 * z / (2 * tau1 * beta + 1), 0)

        # Update v
        v_tilde = (1 - tau2 * t) * v + tau2 * t * Q @ w_new - tau2 * lam
        v_tilde_squared = np.clip(v_tilde**2, a_min=0, a_max=1e6)  # Clip to prevent overflow
        v_new = (v_tilde + np.sqrt(v_tilde_squared + 4 * alpha * tau2)) / 2

        # Update lambda
        lam += t * (Q @ w_new - v_new)

        # Check convergence
        rp = np.linalg.norm(t * Q.T @ (v_new - v), ord=2)
        rd = np.linalg.norm(Q @ w_new - v_new, ord=2)
        if rp < tol and rd < tol:
            break

        if np.any(np.isnan(w_new)) or np.any(np.isinf(w_new)) or np.any(np.isnan(v_new)) or np.any(np.isinf(v_new)):
            print("Numerical instability detected. Stopping iteration.")
            break

        w, v = w_new, v_new

    return w

# Evaluate ADMM for N=30 and N=100
for N in [30, 100]:
    m = int(N * (N - 1) / 2)
    Q = np.random.rand(N, m)  # Generate sparse binary matrix Q
    Q = Q / np.linalg.norm(Q, ord=2)  # Normalize Q to prevent overflow
    Z = np.random.randn(N, N)
    Z = (Z + Z.T) / 2  # Ensure Z is symmetric
    z = extract_upper_triangular(Z)
    z = z / np.linalg.norm(z, ord=2)  # Normalize z to prevent overflow
    start_time = time.time()
    w_admm = admm_graph_learning(z, Q, alpha, beta, rho=1.0)
    admm_time = time.time() - start_time
    print(f"ADMM solution time (N={N}): {admm_time:.4f} seconds")


                                     CVXPY                                     
                                     v1.6.0                                    
(CVXPY) Jan 04 03:19:50 AM: Your problem has 435 variables, 435 constraints, and 0 parameters.
(CVXPY) Jan 04 03:19:50 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 04 03:19:50 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jan 04 03:19:50 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jan 04 03:19:50 AM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jan 04 03:19:50 AM: Compiling problem (target solver=CLARABEL

C:\Users\Password Please\AppData\Local\Temp\ipykernel_4880\2511696383.py:54: RuntimeWarning: overflow encountered in square
  v_tilde_squared = np.clip(v_tilde**2, a_min=0, a_max=1e6)  # Clip to prevent overflow


ADMM solution time (N=30): 0.8902 seconds
ADMM solution time (N=100): 7.3441 seconds


In [10]:
import numpy as np
import scipy.linalg as la
import cvxpy as cp
import time

# Problem setup for Graph Learning
np.random.seed(0)
N = 30  # Number of vertices (changeable for testing)
m = int(N * (N - 1) / 2)  # Number of upper triangular elements in W
Z = np.random.randn(N, N)
Z = (Z + Z.T) / 2  # Symmetric matrix
alpha, beta = 0.1, 0.01

# Flatten upper triangular part of Z excluding diagonal
def extract_upper_triangular(Z):
    return Z[np.triu_indices_from(Z, k=1)]

z = extract_upper_triangular(Z)

# Normalize z to prevent overflow
z = z / np.linalg.norm(z, ord=2)

# Part 1: Solve using CVXPY
start_time = time.time()
w = cp.Variable(m)
Q = np.random.rand(N, m)  # Sparse binary matrix with Qw = W1
Q = Q / np.linalg.norm(Q, ord=2)  # Normalize Q to prevent overflow
loss = 2 * z.T @ w - alpha * cp.sum(cp.log(Q @ w + 1e-6)) + beta * cp.norm(w, 2)**2  # Add small constant to avoid log(0)
constraints = [w >= 0]
problem = cp.Problem(cp.Minimize(loss), constraints)
problem.solve(verbose=True)
cvxpy_time = time.time() - start_time
print(f"CVXPY solution time (N={N}): {cvxpy_time:.4f} seconds")

# Part 2: ADMM Algorithm

def admm_graph_learning(z, Q, alpha, beta, rho, max_iters=1000, tol=1e-6):
    m = len(z)
    s = Q.shape[0]
    w = np.zeros(m)
    v = np.ones(s)
    lam = np.zeros(s)
    t = 0.1  # Reduced step size for stability
    tau1 = 1 / (2 * (N - 1) * t)
    tau2 = 1 / t

    for k in range(max_iters):
        # Update w
        w_tilde = w - tau1 * t * Q.T @ (Q @ w - v - lam / t)
        w_new = np.maximum(w_tilde - 2 * tau1 * z / (2 * tau1 * beta + 1), 0)

        # Update v
        v_tilde = (1 - tau2 * t) * v + tau2 * t * Q @ w_new - tau2 * lam
        v_tilde = np.clip(v_tilde, a_min=-1e3, a_max=1e3)  # Clip directly to prevent overflow
        v_tilde_squared = v_tilde**2
        v_new = (v_tilde + np.sqrt(v_tilde_squared + 4 * alpha * tau2)) / 2

        # Update lambda
        lam += t * (Q @ w_new - v_new)

        # Check convergence
        rp = np.linalg.norm(t * Q.T @ (v_new - v), ord=2)
        rd = np.linalg.norm(Q @ w_new - v_new, ord=2)
        if rp < tol and rd < tol:
            break

        if np.any(np.isnan(w_new)) or np.any(np.isinf(w_new)) or np.any(np.isnan(v_new)) or np.any(np.isinf(v_new)):
            print("Numerical instability detected. Stopping iteration.")
            break

        w, v = w_new, v_new

    return w

# Evaluate ADMM for N=30 and N=100
for N in [30, 100]:
    m = int(N * (N - 1) / 2)
    Q = np.random.rand(N, m)  # Generate sparse binary matrix Q
    Q = Q / np.linalg.norm(Q, ord=2)  # Normalize Q to prevent overflow
    Z = np.random.randn(N, N)
    Z = (Z + Z.T) / 2  # Ensure Z is symmetric
    z = extract_upper_triangular(Z)
    z = z / np.linalg.norm(z, ord=2)  # Normalize z to prevent overflow
    start_time = time.time()
    w_admm = admm_graph_learning(z, Q, alpha, beta, rho=1.0)
    admm_time = time.time() - start_time
    print(f"ADMM solution time (N={N}): {admm_time:.4f} seconds")


                                     CVXPY                                     
                                     v1.6.0                                    
(CVXPY) Jan 04 03:20:54 AM: Your problem has 435 variables, 435 constraints, and 0 parameters.
(CVXPY) Jan 04 03:20:54 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 04 03:20:54 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jan 04 03:20:54 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jan 04 03:20:54 AM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jan 04 03:20:54 AM: Compiling problem (target solver=CLARABEL

In [8]:
import cvxpy as cp
import numpy as np
import time
import pandas as pd

def solve_graph_learning_cvxpy(N, alpha=1.0, beta=1.0):
    """
    Solve the graph learning problem using CVXPY for a graph with N vertices.
    Args:
        N (int): Number of vertices in the graph.
        alpha (float): Regularization parameter for the log barrier term.
        beta (float): Regularization parameter for the Frobenius norm.
    Returns:
        objective_value (float): Final objective value.
        time_taken (float): Time taken to solve the problem.
    """
    np.random.seed(42)
    Z = np.random.randn(N, N)  # Random observation matrix
    Z = (Z + Z.T) / 2  # Symmetrize Z

    # Define the optimization variable
    W = cp.Variable((N, N), symmetric=True)

    # Define the objective function
    objective = cp.Minimize(cp.norm1(cp.multiply(W, Z)) - alpha * cp.sum(cp.log(W @ np.ones(N))) + (beta / 2) * cp.norm(W, 'fro')**2)

    # Define the constraints
    constraints = [W >= 0, cp.diag(W) == 0]

    # Define and solve the problem
    problem = cp.Problem(objective, constraints)
    start_time = time.time()
    problem.solve()
    end_time = time.time()
    
    return problem.value, end_time - start_time

# Test the function for N = 30 and N = 100
graph_sizes = [30, 100]
results = []

for N in graph_sizes:
    obj_value, time_taken = solve_graph_learning_cvxpy(N)
    results.append({
        "graph_size": N,
        "objective_value": obj_value,
        "time_taken_sec": time_taken
    })

# Display the results in a table
df = pd.DataFrame(results)
print(df)


   graph_size  objective_value  time_taken_sec
0          30        -7.479221        0.049864
1         100       -64.834648        0.559603


In [10]:

def admm_graph_learning(N, alpha=1.0, beta=1.0, rho=1.0, max_iter=1000, tol=1e-4):
    """
    ADMM algorithm for graph learning.
    Args:
        N (int): Number of vertices in the graph.
        alpha (float): Regularization parameter for the log barrier term.
        beta (float): Regularization parameter for the Frobenius norm.
        rho (float): ADMM parameter.
        max_iter (int): Maximum number of iterations.
        tol (float): Tolerance for convergence criteria.
    Returns:
        objective_value (float): Final objective value.
        iterations (int): Number of iterations performed.
        time_taken (float): Time taken to solve the problem.
    """
    np.random.seed(42)
    Z = np.random.randn(N, N)
    Z = (Z + Z.T) / 2  # Symmetrize Z

    # Initialize variables
    W = np.random.rand(N, N)
    W = (W + W.T) / 2
    np.fill_diagonal(W, 0)
    v = np.zeros_like(W)
    lam = np.zeros_like(W)

    start_time = time.time()
    
    for k in range(max_iter):
        # Update W using proximal operator (soft-thresholding and non-negativity constraint)
        W_new = np.maximum(W - rho * (W - v - lam / rho), 0)
        np.fill_diagonal(W_new, 0)
        
        # Update v using proximal operator for log barrier term
        v_new = np.maximum(W_new + lam / rho, 1e-6)
        np.fill_diagonal(v_new, 0)
        
        # Update lambda (dual variable)
        lam_new = lam + rho * (W_new - v_new)
        
        # Compute residuals
        r_p = np.linalg.norm(W_new - v_new, 'fro')
        r_d = np.linalg.norm(v_new - v, 'fro')
        
        # Check convergence
        if r_p < tol and r_d < tol:
            break
        
        # Update variables for next iteration
        W, v, lam = W_new, v_new, lam_new
    
    end_time = time.time()
    objective_value = np.sum(np.abs(W_new * Z)) - alpha * np.sum(np.log(np.sum(W_new, axis=1) + 1e-6)) + (beta / 2) * np.linalg.norm(W_new, 'fro')**2
    time_taken = end_time - start_time
    
    return objective_value, k + 1, time_taken

# Test the ADMM algorithm for graph sizes N = 30 and N = 100
graph_sizes = [30, 100]
results = []

for N in graph_sizes:
    obj_value, iterations, time_taken = admm_graph_learning(N)
    results.append({
        "graph_size": N,
        "objective_value": obj_value,
        "iterations": iterations,
        "time_taken_sec": time_taken
    })

# Display the results in a table
pd.DataFrame(results)


,graph_size,objective_value,iterations,time_taken_sec
0,30,414.465317,1,0.0
1,100,1381.551056,1,0.0
